In [73]:
import numpy as np
import copy
import math

In [7]:
# data reader from csv file
def load_data(path, dataType, skip_header):
    data = np.genfromtxt(path, delimiter=',', skip_header=skip_header, dtype=dataType)
    return data 

In [40]:
# the data columns of the data file 
dataType = [
    ('name', 'U40'),
    ('year', int),
    ('selling_price', int),
    ('km_driven', int),
    ('fuel', 'U15'),
    ('seller_type', 'U16'),
    ('transmission', 'U15'),
    ('owner', 'U20')
]

In [41]:
# load the data 
data = load_data('car_data.csv', dataType, True)

In [42]:
print(len(data['name'])) #the name is not that valuable tho
print(len(data['year']))
print(len(data['selling_price']))
print(len(data['km_driven']))
print(len(data['fuel']))
print(len(data['seller_type']))
print(len(data['transmission']))
print(len(data['owner']))


4340
4340
4340
4340
4340
4340
4340
4340


In [49]:
# the categorical features should be encoded to make use of them
seller_type_categories = ["Individual", "Dealer", "Trustmark Dealer"]
fuel_categories = ["Petrol", "Diesel", "CNG", "Electric", "LPG"]
transmission_categories = ["Manual", "Automatic"]
owner_categories = ["First Owner", "Second Owner", "Third Owner", "Fourth & Above Owner"]

In [52]:
# I used a integers to label the different categories
seller_type_mapping = {
    "Individual" : 1,
    "Dealer" : 2,
    "Trustmark Dealer" : 3
}

fuel_mapping = {
    "Petrol" : 1,
    "Diesel" : 2,
    "CNG" : 3,  # CNG means Compressed Natural Gas
    "Electric" : 4, 
    "LPG" : 5   # LPG means Liquefied Petroleum Gas
}
transmission_mapping = {
    "Manual" : 1,
    "Automatic" : 2
}
owner_mapping = {
    "First Owner" : 1,
    "Second Owner" : 2,
    "Third Owner" : 3,
    "Fourth & Above Owner" : 4,
    "Test Drive Car" : 5
}

In [112]:
# These 6 are the ones to be used as a feature
seller_type = np.array([seller_type_mapping[s_type] for s_type in data['seller_type']])
fuel = np.array([fuel_mapping[f_type] for f_type in data['fuel']])
transmission = np.array([transmission_mapping[t_type] for t_type in data['transmission']])
owner = np.array([owner_mapping[o_type] for o_type in data['owner']])
age = np.array([(2023-year) for year in data['year']])
km_driven = np.array(data['km_driven'])


In [127]:
# writing the X, the features matrix
x_train = np.column_stack((seller_type, fuel, transmission, owner, age, km_driven))
Y = np.array(data['selling_price'])

In [153]:
# feature scaling using zscore normalization
def zscore_normalize_features(X):
    # find the mean of each column/feature
    mu     = np.mean(X, axis=0)                 # mu will have shape (n,)
    # find the standard deviation of each column/feature
    sigma  = np.std(X, axis=0)                  # sigma will have shape (n,)
    # element-wise, subtract mu for that column from each example, divide by std for that column
    X_norm = (X - mu) / sigma      

    return (X_norm, mu, sigma)

In [155]:
# normalize the original features
X_norm, X_mu, X_sigma = zscore_normalize_features(x_train)
print(f"X_mu = {X_mu}, \nX_sigma = {X_sigma}")

X_mu = [1.27603687e+00 1.53640553e+00 1.10322581e+00 1.46658986e+00
 9.90921659e+00 6.62157774e+04], 
X_sigma = [4.96835107e-01 5.76267972e-01 3.04253577e-01 7.40244624e-01
 4.21485827e+00 4.66387281e+04]


In [135]:
def compute_cost(X, y, w, b): 
    """
    compute cost
    Args:
      X (ndarray (m,n)): Data, m examples with n features
      y (ndarray (m,)) : target values
      w (ndarray (n,)) : model parameters  
      b (scalar)       : model parameter
      
    Returns:
      cost (scalar): cost
    """
    m,n = X.shape
    # calculate f_wb for all examples.
    f_wb = X @ w + b  
    # calculate cost
    total_cost = (1/(2*m)) * np.sum((f_wb-y)**2)

    # if verbose: print("f_wb:")
    # if verbose: print(f_wb)
        
    return total_cost

In [158]:
print(compute_cost(X_norm, Y, [1, 2, 3, 4, 5, 6], 1))


294393448223.70355


In [184]:
# function to compute the gradients
def compute_gradient_matrix(X, y, w, b): 
    
    m,n = X.shape
    f_wb = X @ w + b              
    e   = f_wb - y                
    dj_dw  = (1/m) * (X.T @ e)    
    dj_db  = (1/m) * np.sum(e)    
    # print(f"the gradient {(dj_db,dj_dw)}")
    return dj_db,dj_dw

In [138]:
#Compute and display gradient 
# tmp_dj_db, tmp_dj_dw = compute_gradient_matrix(X_norm, Y, [1,2,3,4,5,6], 1)
# print(f'dj_db at initial w,b: {tmp_dj_db}')
# print(f'dj_dw at initial w,b: \n {tmp_dj_dw}')

In [144]:
def gradient_descent(X, y, w_in, b_in, cost_function, gradient_function, alpha, num_iters): 
    
    # number of training examples
    m = len(X)
        
    w = copy.deepcopy(w_in)  #avoid modifying global w within function
    b = b_in

    for i in range(num_iters):

        # Calculate the gradient and update the parameters
        dj_db,dj_dw = gradient_function(X, y, w, b)   

        # Update Parameters using w, b, alpha and gradient
        w = w - alpha * dj_dw               
        b = b - alpha * dj_db               
      
    return w, b

In [189]:
# initialize parameters
initial_w = np.zeros(6)
initial_b = 0.
# some gradient descent settings
iterations = 30000
alpha = 1
# run gradient descent 
w_final, b_final = gradient_descent(X_norm, Y, initial_w, initial_b,
                                                    compute_cost_matrix, compute_gradient_matrix, 
                                                    alpha, iterations)
print(f"b,w found by gradient descent: {b_final:0.2f},{w_final} ")

b,w found by gradient descent: 504127.31,[  47742.63401378  109531.30277309  266588.84953759   -7784.96183229
 -169386.44666937  -26234.31006287] 


In [190]:
# predictor
def predict(car_data)->float:
    car_norm = (car_data - X_mu) 
    # print(X_mu)
    # print(car_norm)
    car_predict = np.dot(w_final, car_norm) + b_final
    print(car_predict)

In [191]:
predict([1, 1, 1, 3, 2000, 100000])

-1223007437.7650487
